# Linear regression 

In this exercise you will use linear regression to predict flat prices. One more time, training will be handled via gradient descent. Although, contratry to the first exercise, we will now:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a different loss function.

Let's start with getting the data.

In [ ]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2021-03-14 14:37:50--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2021-03-14 14:37:50--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8c6096aaff0d54ca28901901cb.dl.dropboxusercontent.com/cd/0/get/BKp0-eDEB7J9Wtu9ofUNYpsfkfhLfDfngotLEXnClqFF54l5vHXEvnN_wCBcfjr5oDeGsvw1Y-r3_4tXB_ZHvFYCzu6r7_5GEiIQL5H9ZkBJ73iacVtFJh-RKtFHA4AgAHVKk0Yq2eOduzCVWGN39uwh/file?dl=1# [following]
--2021-03-14 14:37:50--  https://uc8c6096aaff0d54ca28901901cb.dl.dropboxusercontent.com/cd/0/get/BKp0-eDEB7J9Wtu9ofUNYpsfkfhLfDfngotLEXnClqFF54l5vHXEvnN_wCBcfjr5oDeGsvw1Y-r3_4tXB_

In [ ]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


In [13]:
import pandas as pd
import numpy as np

train = pd.read_csv('mieszkania.csv')
test = pd.read_csv('mieszkania_test.csv')

total = pd.concat([train, test])

total = pd.get_dummies(total, prefix='dzielnica')

train = total[:len(train)]
test = total[len(train):]

train.head()

,m2,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena,dzielnica_grodziskowo,dzielnica_mokotowo,dzielnica_ochotowo,dzielnica_wolowo
0,104,2,2,1940,1,780094,0,1,0,0
1,43,1,1,1970,1,346912,0,0,1,0
2,128,3,2,1916,1,523466,1,0,0,0
3,112,3,2,1920,1,830965,0,1,0,0
4,149,3,3,1977,0,1090479,0,1,0,0


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`). 

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [ ]:
def msle(ys, ps):
    assert len(ys) == len(ps)
    #################################
    # TODO: Implement this function #
    #################################
    sum = 0
    for i in range(len(ys)):
      sum += (np.log(1+ys[i]) - np.log(1+ps[i]))**2
    return (1/len(ys)) * sum

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [24]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################

mean_price = np.mean(train['cena'])
ps = []
for i in range(len(train['cena'])):
  ps.append(mean_price)
msle(test['cena'], ps)


0.4284115392580848

Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [25]:
#############################################
# TODO: Find this constant and compute msle #
#############################################
best_const = np.exp(np.mean(np.log(1 + train['cena']))) -1
ps2 = []
for i in range(len(train['cena'])):
  ps2.append(best_const)
msle(test['cena'], ps2)


0.42757520337601745

Now, let's implement a standard linear regression model. 

In [26]:
def generate(train, test):
  train_y = train['cena'].to_numpy()
  test_y = test['cena'].to_numpy()
  n=len(test_y)

  train_x = train.drop('cena', axis = 1)
  test_x = test.drop('cena', axis = 1)
  
  mean, std = train_x.mean().to_numpy(), train_x.std().to_numpy()

  train_x, test_x = train_x.to_numpy(), test_x.to_numpy()

  train_x = (train_x - mean) / std
  train_x = np.hstack((np.ones_like(train_x), train_x)) # bias

  test_x = (test_x - mean) / std
  test_x = np.hstack((np.ones_like(test_x), test_x)) # bias
  return train_x, train_y, test_x, test_y, n

  
train_x, train_y, test_x, test_y, n = generate(train, test)


In [28]:
#########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################

lr = 0.001

def evaluate(ws, xs, ys):
  ps = np.dot(xs, ws)
  assert len(ys) == len(ps)
  return np.mean((ys - ps)**2)

def fit(x, y, steps):
  print(x.shape[1])
  n = len(x)
  W = np.random.normal(size=x.shape[1])
  for i in range(steps):
    preds = np.dot(x, W)
    delta = preds - y

    gradient = (2/n)*np.dot(x.T, delta)
    W -= lr*gradient

    if i % 10_000 == 0:
      loss = evaluate(W, x, y)
      print('Iter: {:> 3} Loss: {:8.8f}'.format(i, loss))
  return W


In [29]:
W=fit(train_x, train_y, 100000)
W_l=fit(train_x, np.log1p(train_y), 100000)

18
Iter:   0 Loss: 321878871823.50488281
Iter:  10000 Loss: 4158729180.25534678
Iter:  20000 Loss: 3910095720.53068590
Iter:  30000 Loss: 3874334460.64409685
Iter:  40000 Loss: 3869190081.31923294
Iter:  50000 Loss: 3868450044.70346785
Iter:  60000 Loss: 3868343587.89728355
Iter:  70000 Loss: 3868328273.72046995
Iter:  80000 Loss: 3868326070.72359276
Iter:  90000 Loss: 3868325753.81493425
18
Iter:   0 Loss: 130.24993351
Iter:  10000 Loss: 0.03316841
Iter:  20000 Loss: 0.03066548
Iter:  30000 Loss: 0.03030677
Iter:  40000 Loss: 0.03025516
Iter:  50000 Loss: 0.03024774
Iter:  60000 Loss: 0.03024667
Iter:  70000 Loss: 0.03024652
Iter:  80000 Loss: 0.03024650
Iter:  90000 Loss: 0.03024649


In [30]:
pred = test_x @ W
print(f'Score after regression = {msle(test_y, pred)}')
print(list(zip(pred[:10], test_y[:10])))

Score after regression = 0.18097954553844425
[(377531.00737974106, 322227), (371324.00755755854, 295878), (363304.24029653566, 306530), (552899.7397888, 553641), (961779.6817559316, 985348), (757216.1745830548, 695726), (1711.421503894133, 99751), (814211.6758681274, 891261), (538791.6708954377, 536499), (637412.9729568154, 527093)]


Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint: 
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [31]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################
pred=np.expm1(test_x @ W_l)
print(f'Score after regression = {msle(test_y, pred)}')
print(list(zip(pred[:10], test_y[:10])))


Score after regression = 0.03656900112944419
[(325343.513758225, 322227), (319965.65787188127, 295878), (296897.3704915062, 306530), (424583.43319257454, 553641), (1198597.7101786844, 985348), (801489.4544147662, 695726), (141145.57574709304, 99751), (860735.3325141784, 891261), (447010.5851227259, 536499), (618671.0367028143, 527093)]


Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code. 

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint: 
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2: 
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [38]:
###############################################
# TODO: Implement the feature engieering part #
###############################################

cols = [x for x in train.columns if x != 'cena']

def mul_cols(df):
  a,b = df.shape
  ndf = np.zeros(shape=(a,int(b*(b+1)/2)))
  k=0
  for i in range(b):
    for j in range(i, b):
      ndf[:, k] = df[:, i]*df[:, j]
      k+=1
  return ndf

train_x_new = mul_cols(train_x)
test_x_new = mul_cols(test_x)

In [39]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################

W_l=fit(train_x_new, np.log1p(train_y), 100000)
pred=np.expm1(test_x_new @ W_l)
print(f'Score after regression = {msle(test_y, pred)}')
print(list(zip(pred[:10], test_y[:10])))

171
Iter:   0 Loss: 342.04634694
Iter:  10000 Loss: 0.05612666
Iter:  20000 Loss: 0.01353222
Iter:  30000 Loss: 0.00746967
Iter:  40000 Loss: 0.00648294
Iter:  50000 Loss: 0.00628472
Iter:  60000 Loss: 0.00622900
Iter:  70000 Loss: 0.00620527
Iter:  80000 Loss: 0.00619128
Iter:  90000 Loss: 0.00618156
Score after regression = 0.01991990533533927
[(430125.28540649015, 322227), (338968.5396032083, 295878), (289609.2893225403, 306530), (509430.55165589397, 553641), (969456.1291683296, 985348), (681399.5148455193, 695726), (116642.79676230457, 99751), (873781.6549954314, 891261), (541268.4677527512, 536499), (527154.1003653996, 527093)]
